In [18]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

import matplotlib.pyplot as plt


In [9]:
from glob import glob

In [2]:
IMAGE_SIZE = [224, 224]

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights=None, include_top=False)


In [5]:
vgg.load_weights("vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [6]:
for layer in vgg.layers:
    layer.trainable = False

In [15]:
folders = glob('dataset/*')
folders


['dataset\\without_mask', 'dataset\\with_mask']

In [16]:
x = Flatten()(vgg.output)

prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)


model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [17]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [19]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [20]:
training_set = train_datagen.flow_from_directory('dataset',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 440 images belonging to 2 classes.


In [21]:
history = model.fit_generator(
  training_set,
   
  epochs=5,
  steps_per_epoch=len(training_set),
  
)

Epoch 1/5
14/14 [==============================] - 260s 19s/step - loss: 0.9527 - accuracy: 0.6864
Epoch 2/5
14/14 [==============================] - 210s 15s/step - loss: 0.1886 - accuracy: 0.9250
Epoch 3/5
14/14 [==============================] - 207s 15s/step - loss: 0.0833 - accuracy: 0.9750
Epoch 4/5
14/14 [==============================] - 210s 15s/step - loss: 0.0351 - accuracy: 0.9909
Epoch 5/5
14/14 [==============================] - 213s 15s/step - loss: 0.0307 - accuracy: 0.9932


In [22]:
model.save('Mask_detector_model.h5')